In [1]:
%load_ext autoreload
%autoreload 2
import  pandas as pd
#import lc_module as lm
import numpy as np
import gc
import sys
import time
from pathlib import Path

import dask
from dask.distributed import Client
import dask.dataframe as dd

In [2]:
# You need to start the cluster elsewhere.
client = Client('tls://192.168.102.39:44701')

In [3]:
# Lightcone configuration. 
BoxLength = 2000.
## compute theta_max given a square_coverage_area
Area_in_square_degrees = 50 ## A is the area coverage in square degrees that corresponds to the Euclid Deep Survey

# define the choose direction as a vector
ra = np.radians(31) 
dec  = np.radians(10)
box_info = {"Box1":[0,0,0],"Box2":[BoxLength,0,0],"Box3":[BoxLength,BoxLength,0],
            "Box4":[2*BoxLength,BoxLength,0]}

### Step 1: Read the table to make the JOIN
df = pd.read_pickle('../join_table.pkl')
path_to_files = '/data/astro/scratch/sramakri/uchuu_selected_region/'

jointable = df

In [4]:
#import lc_module as lm
import lightcone as lightcone

updated


In [5]:
@dask.delayed
def wrapper(i):
    sys.path.append('/nfs/pic.es/user/s/sramakri/jpynb/lc_interpolator_dask_version')
    import lightcone as lightcone
    res = lightcone.gen_slice(i,BoxLength,Area_in_square_degrees,ra,dec,box_info,df,path_to_files)
    #res['slice'] = i
    return res

In [6]:
# Specifying the output columns upfront.
meta = pd.DataFrame(columns=['id','pid','upid','scale_of_last_mm', 'x', 'y', 'z','xoff','vx','vy','vz','c_to_a','b_to_a','vrms','vmax','vpeak','a_x','a_y','a_z','jx','jy','jz','spin','rs','rvir','rvmax','m200b','mvir','m200c'], \
                    dtype=np.float32)
meta['id'] = meta.id.astype(np.int64)
meta['pid'] = meta.pid.astype(np.int64)
meta['upid'] = meta.upid.astype(np.int64)

In [7]:
meta.keys

<bound method NDFrame.keys of Empty DataFrame
Columns: [id, pid, upid, scale_of_last_mm, x, y, z, xoff, vx, vy, vz, c_to_a, b_to_a, vrms, vmax, vpeak, a_x, a_y, a_z, jx, jy, jz, spin, rs, rvir, rvmax, m200b, mvir, m200c]
Index: []

[0 rows x 29 columns]>

In [28]:
L = [dd.from_delayed(wrapper(i), meta=meta) for i in range(92)]
#L = [dd.from_delayed(wrapper(i), meta=meta) for i in range(3)]

In [29]:
res = dd.concat(L)

In [30]:
d_out = Path('/data/astro/scratch/sramakri/lc/')

In [31]:
t1 = time.time()
res.to_parquet(d_out / 'lc_v13.pq')
print('time', time.time() - t1)

time 742.4914708137512
